In [ ]:
# Load the required libraries
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import KFold
import plotly.graph_objects as go

In [ ]:
# Define the stock symbol and timeframe for the data
symbol = 'FTM-USD'
start_date = '2018-10-30'
end_date = '2023-05-03'

# Retrieve the stock data from Yahoo Finance
stock_data = yf.download(symbol, start=start_date, end=end_date)

# Create a new dataframe with only the 'Close' column
data = stock_data.filter(['Close'])

In [ ]:
# Convert the dataframe to a numpy array
dataset = data.values

# Get the number of rows to train the model on
training_data_len = int(np.ceil(len(dataset) * .8))

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Define the number of folds
n_splits = 5

# Create an instance of the KFold class
kf = KFold(n_splits=n_splits, shuffle=True)

# Initialize the array to store the fold metrics
fold_rmse_values = np.zeros(n_splits)

# Loop through each fold and train the model
for fold, (train_index, val_index) in enumerate(kf.split(scaled_data)):
    # Split the data into training and validation sets for this fold
    train_data = scaled_data[train_index]
    val_data = scaled_data[val_index]

    # Split the data into x_train and y_train datasets
    x_train = []
    y_train = []

    lookback = 60
    for i in range(lookback, len(train_data)):
        x_train.append(train_data[i-lookback:i, 0])
        y_train.append(train_data[i, 0])

    # Convert the x_train and y_train to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Build the CNN model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # Compile the model
    optimizer = RMSprop(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    model.fit(x_train, y_train, batch_size=1, epochs=50, verbose=1)

    # Create the testing dataset
    test_data = scaled_data[training_data_len - lookback:, :]

    # Split the data into x_test and y_test datasets
    x_test = []
    y_test = dataset[training_data_len:, :]

    for i in range(lookback, len(test_data)):
        x_test.append(test_data[i-lookback:i, 0])

    # Convert the data to a numpy array
    x_test = np.array(x_test)

    # Reshape the data
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Get the model's predicted price values for the test dataset
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    # Get the root mean squared error (RMSE)
    rmse = np.sqrt(np.mean(predictions - y_test) ** 2)
    print(f"Fold {fold+1} RMSE: {rmse}")

    # Store the RMSE value for this fold
    fold_rmse_values[fold] = rmse

# Calculate the average RMSE across all folds
average_rmse = np.mean(fold_rmse_values)
print(f"Average RMSE: {average_rmse}")

# Train the final model on all the training data
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=100, batch_size=32)

# Make a prediction for the next day's closing price
last_date = stock_data.index[-1]
next_date = last_date + pd.DateOffset(days=1)
last_price = data['Close'].iloc[-lookback:].values.reshape(-1, 1)
last_price_scaled = scaler.transform(last_price)
next_day_prediction_scaled = model.predict(last_price_scaled.reshape(1, lookback, 1))[0][0]
next_day_prediction = scaler.inverse_transform([[next_day_prediction_scaled]])[0][0]
print(f'Predicted Close Price for {next_date.date()}: {next_day_prediction:.2f}')


In [ ]:
# Plot the actual and predicted prices
fig1 = go.Figure()

# Add actual price trace for training dataset
fig1.add_trace(go.Scatter(x=stock_data[:training_data_len].index, y=stock_data['Close'][:training_data_len], mode='lines', name='Actual (Train)'))

# Add actual price trace for test dataset
fig1.add_trace(go.Scatter(x=stock_data[training_data_len:].index, y=stock_data['Close'][training_data_len:], mode='lines', name='Actual (Test)'))

# Add predicted price trace
fig1.add_trace(go.Scatter(x=stock_data[training_data_len:].index, y=predictions[:, 0], mode='lines', name='Predicted'))

# Set the layout and titles
fig1.update_layout(title='Actual vs. Predicted Stock Prices', xaxis_title='Date', yaxis_title='Price')

# Show the plot
fig1.show()

# Plot the predicted and actual closing prices with the next day's prediction
fig2 = go.Figure()

# Add actual closing price trace
fig2.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Actual Closing Prices'))

# Add predicted closing price point for the next day
fig2.add_trace(go.Scatter(x=[next_date], y=[next_day_prediction], mode='markers', marker=dict(size=10), name='Predicted Closing Price'))

# Set the layout and titles
fig2.update_layout(title=f'Predicted Closing Price for {next_date.date()}: {next_day_prediction:.2f}', xaxis_title='Date', yaxis_title='Closing Price')

# Show the plot
fig2.show()

# Create a bar chart to visualize the predicted closing price for the next day
fig3 = go.Figure()

# Add the last closing price bar
fig3.add_trace(go.Bar(x=[last_date], y=[stock_data['Close'][-1]], name='Last Close Price'))

# Add the predicted closing price bar
fig3.add_trace(go.Bar(x=[next_date], y=[next_day_prediction], name='Predicted Next Close Price'))

# Set the layout and titles
fig3.update_layout(title=f'Last Closing Price vs Predicted Next Closing Price', xaxis_title='Date', yaxis_title='Closing Price')

# Show the plot
fig3.show()
